## Pytorch CNN 
- Fashion MNIST
- 컨볼루션을 거쳐 만들어진 새로운 이미지는 특징 맵 (Feature map)이라고 불림
- 특징 맵의 크기가 크면 학습이 어렵고 과적합의 위험이 증가
- 

In [15]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torchvision import transforms, datasets

USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")

epochs = 10
batch_size = 64

train_loader = torch.utils.data.DataLoader(
    datasets.FashionMNIST('/data', train=True, download=True,
                          transform = transforms.Compose([
                              transforms.ToTensor(),
                              transforms.Normalize((0.1307,), (0.3081,))
                          ])),
    batch_size = batch_size, shuffle = True
)

test_loader = torch.utils.data.DataLoader(
    datasets.FashionMNIST('/data', train=False, download = True,
                         transform = transforms.Compose([
                             transforms.ToTensor(),
                             transforms.Normalize((0.1307, ), (0.3081,))
                         ])),
    batch_size = batch_size, shuffle = True
)

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5) # input 채널수, output 채널수
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5) # 10개의 특징맵 , 20개의 특징맵
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)
        
    # training 용
    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2)) # C -> M
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2)) # C -> D -> M
        x = x.view(-1, 320) # reshape
        x = F.relu(self.fc1(x)) # f - relu
        x = F.dropout(x, training = self.training)
        x = self.fc2(x) 
        return F.log_softmax(x, dim=1)

def train(model, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(DEVICE), target.to(DEVICE) # data, label DEVICE 로 DATA 보내기

        # 반복 때마다 기울기를 새로 계산하므로, optimizer.zero_grad() 함수 호출
        optimizer.zero_grad() # 최적화 함수에 대한 미분 진행 여부 설정
        output = model(data) # model 통과한 결과
        loss = F.cross_entropy(output, target) # target 과의 비교를 통한 cross entropy
        loss.backward() # 역방향 전파
        optimizer.step() # 미분 반복 진행
        
        if batch_idx % 200==0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(epoch, batch_idx*len(data),
                                                                           len(train_loader.dataset),
                                                                           100.*batch_idx / len(train_loader), loss.item())
                 )

# 평가때는 기울기를 계산하지 않음
def evaluate(model, test_loader):
    model.eval() # 평가 모드로 모델 변경
    test_loss = 0 # 테스트 오차
    correct=0 # 예측이 맞은 수
    
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE) # test data, target DEVICE 로 전송
            output = model(data) 
            
            # 배치 오차 합산하기
            # 교차엔트로피시 미니배치의 합을 받아와야 함. 
            test_loss += F.cross_entropy(output, target, reduction='sum').item() # item() 함수 - 1개의 값을 가진 텐서의 값을 가져옴
        
            # 가장 높은 값을 가진 인덱스가 바로 예측값
            pred = output.max(1, keepdim=True)[1] # 가장 큰 확률 값을 가진 값 예측 -> 가장 큰 값과, 해당 값의 index 반환 -> index 사용
            correct += pred.eq(target.view_as(pred)).sum().item() # view_as => target을 pred의 shape처럼 바꿔줌 / sum().item() => 맞춘갯수의 합

    # 정확도 구하기
    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)

    return test_loss, test_accuracy 


In [17]:
# to 함수는 모델의 파라미터들을 지정한 장치의 메모리로 보냄
model = Net().to(DEVICE)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

# 실제 학습 진행
for epoch in range(1, epochs+1):
    train(model, train_loader, optimizer, epoch)
    test_loss, test_accuracy = evaluate(model, test_loader)
    
    print('[{}] Test Loss: {:.4f}, Accuracy: {:.2f}%'.format(epoch, test_loss, test_accuracy))

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.402542
Train Epoch: 1 [12800/60000 (21%)]	Loss: 1.442082
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.964541
Train Epoch: 1 [38400/60000 (64%)]	Loss: 1.036943
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.927286
[1] Test Loss: 0.6932, Accuracy: 73.36%
Train Epoch: 2 [0/60000 (0%)]	Loss: 0.814311
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.552045
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.793093
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.747909
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.661036
[2] Test Loss: 0.6094, Accuracy: 76.58%
Train Epoch: 3 [0/60000 (0%)]	Loss: 0.704861
Train Epoch: 3 [12800/60000 (21%)]	Loss: 0.731342
Train Epoch: 3 [25600/60000 (43%)]	Loss: 0.697576
Train Epoch: 3 [38400/60000 (64%)]	Loss: 0.614212
Train Epoch: 3 [51200/60000 (85%)]	Loss: 0.654692
[3] Test Loss: 0.5563, Accuracy: 78.32%
Train Epoch: 4 [0/60000 (0%)]	Loss: 0.611713
Train Epoch: 4 [12800/60000 (21%)]	Loss: 0.599469
Train Epoch: 4 [25600/60000 (43%)]	Loss: 0.864300
